In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd

from scipy.spatial import KDTree, cKDTree
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import time
import joblib
from tqdm import tqdm
import trackpy as tp
from numba import njit, prange

from yupi import Trajectory
import yupi.graphics as yg
import yupi.stats as ys

from utility import get_imsd, get_imsd_windowed, get_emsd, get_emsd_windowed, fit_hist, MB_2D,\
                    normal_distr, lorentian_distr, get_trajs, speed_windowed, theta_windowed, \
                    get_smooth_trajs, get_velocities
                   
show_verb = True
save_verb = True
anim_show_verb = False

In [2]:
rawTrajs = pd.read_parquet("../tracking/results/parquet/pre_merge_tracking.parquet")
red_particle_idx = 17
rawTrajs.loc[rawTrajs.particle != red_particle_idx, ["color"]] = "#00007F"
rawTrajs.loc[rawTrajs.particle == red_particle_idx, ["color"]] = "#FF0000"
colors = rawTrajs.loc[rawTrajs.frame == 0, 'color'].values
nDrops = len(rawTrajs.loc[rawTrajs.frame==0])
nFrames = max(rawTrajs.frame) + 1
print(f"nDrops:{nDrops}")
print(f"nFrames:{nFrames} --> {nFrames/10:.2f} s")


# WINDOWED ANALYSIS PARAMETERS
window = 3200 # 320 s
stride = 100 # 10 s
print(f"window of {window/10} s, stride of {stride/10} s")
startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"number of steps: {nSteps}")


# step 10 with a 10 fps video --> 1 s
units = "px/s"
default_kwargs_blue = {"color": "#00FFFF", "ec": (0, 0, 0, 0.6), "density": True}
default_kwargs_red = {"color": "#EE4B2B", "ec": (0, 0, 0, 0.6), "density": True}

nDrops:50
nFrames:32000 --> 3200.00 s
window of 320.0 s, stride of 10.0 s
number of steps: 288


In [3]:
# Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)

# MSD

In [ ]:
show_verb = False
run_windowed_analysis = False
animated_plot_verb = False
print(f"MSD Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")

%run ./analysis_modules/msd.ipynb

# SPEED & TURNING ANGLES DISTRIBUTIONS

In [ ]:
show_verb = False
animated_plot_verb = True
print(f"Speed and Turning Angles Analysis: show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/speed_and_turning_angles.ipynb

# VELOCITY AUTOCORRELATION

In [ ]:
show_verb = False
run_windowed_analysis = False
overwrite = False
animated_plot_verb = False

print(f"Velocity Autocorrelation Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_autocorrelation.ipynb

# RADIAL DISTRIBUTON FUNCTION

In [9]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

print("Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/radial_distribution_function.ipynb

Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}
RDF - Raw Trajectories
raw
RDF - Smooth Trajectories
smooth


14:21:12 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:21:12 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1000x400 -pix_fmt rgba -r 120 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./results/radial_distribution_function/rdf_raw.mp4
14:22:44 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:22:44 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 1000x400 -pix_fmt rgba -r 120 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./results/radial_distribution_function/rdf_smooth.mp4


RDF from center - Raw Trajectories
RDF from center - Smooth Trajectories


14:24:30 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:24:30 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 120 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./results/radial_distribution_function/rdf_from_center_raw.mp4
14:25:52 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
14:25:52 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 640x480 -pix_fmt rgba -r 120 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./results/radial_distribution_function/rdf_from_center_smooth.mp4


# 3 BODY STRUCTURE METRIC

In [ ]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

%run ./analysis_modules/3body_analysis.ipynb